# Задание №1

Запрос написан на PostgreSQL

In [ ]:
query = '''
-- сохраним уроки всех студентов профессии data-analyst страны Serbia апрельской когорты 2022 года
WITH lessons AS 
    (SELECT q.date_created AS lesson_datetime, 
            q.lesson_id,
            q.profession_name,
            q.user_id
     FROM
     -- отберем уроки всех студентов профессии data-analyst страны Serbia, с указанием даты первого урока
         (SELECT f.date_created, 
                 f.lesson_id,
                 l.profession_name,
                 f.user_id,
                 MIN(f.date_created) OVER(PARTITION BY f.user_id) AS first_lesson_datetime
          FROM finished_lesson_test f LEFT JOIN
               lesson_index_test l ON f.lesson_id = l.lesson_id
          WHERE l.country = 'Serbia' AND l.profession_name = 'data-analyst') AS q
     -- оставим только апрельскую когорту    
     WHERE q.first_lesson_datetime BETWEEN '2022-04-01' AND '2022-04-30')
-- напишем основной запрос 
SELECT *
FROM
-- рассчитаем время между уроками
    (SELECT 
           (LEAD(lesson_datetime::timestamp) OVER(PARTITION BY user_id ORDER BY lesson_datetime) - lesson_datetime::timestamp) AS delta_seconds,
           lesson_datetime,
           lesson_id,
           LEAD(lesson_datetime) OVER(PARTITION BY user_id ORDER BY lesson_datetime) AS next_lesson_datetime,
           profession_name,
           user_id
     FROM lessons) AS q2
-- оставим только те уроки где оно меньше либо равно 7 секунд      
WHERE q2.delta_seconds <= '7 seconds'
ORDER BY q2.delta_seconds
'''

# Задание №2

[Ссылка на дашборд в Tableau Public](https://public.tableau.com/views/Dashboard_16784479720840/Dashboard1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link)

# Задание №3

1. Для начала я бы попробовала уточнить у руководителя значение слов "в последнее время" (может это последние полгода, может месяц, а может 2 недели), для того что бы понять за какой промежуток времени мне нужно будет сделать выгрузку. Допустим ответом был бы месяц (если бы не получила ответа, то просто выгрузила бы данные с запасом).
2. После уточнения задачи, я бы выгрузила данные по задачам из Jira с отметками о времени нахождения на каждом этапе за год. 
3. Разбила бы все задачи на когорты по дате формирования. В одну когорту все задачи сформированные на одной неделе.
4. И посмотрела бы на динамику длительности выполнения задач по когортам в разрезе по статусам задач.
5. Исходя из графиков можно было бы сделать выводы, на каком именно этапе внутри цикла происходит увеличение длительности:
    - если в более поздних когортах наблюдается увеличение длительности на этапе In Progress, это может свидетельствовать либо о проблемах в планировании задач, либо о проблемах неэффективности работы сотрудников
    - если в более поздних когортах наблюдается увеличение длительности на этапе Needs to be Clarified, то здесь наверняка проблемы с планированием. Этой метрике я бы уделила особое внимание, потому что, на мой взгяд, в идеале эта метрика должна стремится к нулю, ведь при правильно проведенном PBR не должно быть задач проходящих через этот этап. 